# INSERT INTO DB

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')

In [2]:
reviews = pd.read_json('./prc_data/reviews_prc_3.json')

In [3]:
import sqlite3
conn = sqlite3.connect('./apparel.db')
cur = conn.cursor()

### item table

In [ ]:
items = pd.read_json('./items_prc.json')
items.info()

In [8]:
# 데이터 입력에 필요한 형태로 변환
items = items[['item_id', 'cat', 'name', 'price', 'img', 'link']]
items.item_id = items.item_id.astype(str)

In [9]:
for row in items.itertuples(index=False, name=None):
    cur.execute('''
                INSERT INTO item VALUES (?, ?, ?, ?, ?, ?); ''', row)
    conn.commit()

## reviews => user, review로 나누기

In [4]:
reviews.head()

,품번,아이디,별점,키,몸무게,평소사이즈,리뷰,사이즈,색상
0,550,핫핑-연재,-1,166~170,51 ~ 55,1.0,안사면 손해.. 완전 대만족~! 베이직한 니트 찾으시는 분들 교복템 여기있어요 🙋‍...,None,None
1,550,핫핑-민지,-1,156~160,46 ~ 50,1.0,멋스러운 반팔니트 가을부터 겨울까지 활용 가능한 아이템은 반팔니트가 아닐까 싶어요!...,F,라일락그레이
2,550,150567241,5,171~175,65~,3.0,바지랑입기 딱좋을거 같아서 구매 했는데 너무 예뻐요~ 바지를 더 구매하고싶은 욕구가...,F,크림
3,550,298224934,5,156~160,46 ~ 50,1.0,이런 니트반팔 사고싶었는데 진짜 이뻐요 사이즈도 잘 맞아요 얼른 입고 나가고 싶어요,F,크림
4,550,kies0,5,156~160,56 ~ 60,2.0,도톰하니. 맘에들어요 천도 관리만잘하면 보풀도 잘일어나지않을것같아요 여름끝자락~가을...,F,크림


In [7]:
reviews.replace({np.NaN: None}, inplace=True)

In [8]:
# user, review 정보 나누기
user = reviews[['아이디', '키', '몸무게', '평소사이즈']]
review = reviews[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]

### user Table

In [9]:
for col in ['키', '몸무게', '평소사이즈']:
    print(col)
    print(user[col].unique())

키
['166~170' '156~160' '171~175' '151~155' None '161~165' '146~150' '176이상'
 '139이하' '140~145']
몸무게
['51 ~ 55' '46 ~ 50' '65~' '56 ~ 60' '~45' None '61 ~ 65']
평소사이즈
[1.0 3.0 2.0 None 4.0 5.0]


In [10]:
user_dup = user.loc[user['아이디'].duplicated()].index
user.drop(user_dup, axis=0, inplace=True)
user.reset_index(drop=True, inplace=True)
user.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11159 entries, 0 to 11158
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     11159 non-null  object
 1   키       8261 non-null   object
 2   몸무게     7600 non-null   object
 3   평소사이즈   7559 non-null   object
dtypes: object(4)
memory usage: 348.8+ KB


In [11]:
# 아이디 중복 확인 후 데이터 입력
for row in user.itertuples(name=None, index=False):
    # cur.execute(f'''
    #             INSERT INTO user VALUES(?, ?, ?, ?)
    #             SELECT * FROM user
    #             WHERE NOT EXISTS
    #             (SELECT 1 FROM USER
    #             WHERE id == {row[0]});''', row)
    cur.execute('''
                INSERT OR IGNORE
                INTO user VALUES(?,?,?,?);''', row) # 아이디 중복 제외
    
    conn.commit()

In [12]:
cur.execute('''
            SELECT COUNT(*) FROM user;''')
print(cur.fetchone())

(45565,)


### review Table

In [13]:
review = review[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]
review_dup = review.loc[review['리뷰'].duplicated()==True].index
review.drop(review_dup, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
len(review)

17805

In [14]:
review.duplicated().sum()

0

In [15]:
# '''
# review table 줄바꿈 제거 필요
# '''
# cur.execute('''
#             UPDATE review
#             SET comment = replace(comment, '\n', ' ');''')
# conn.commit()

In [15]:
for row in review.itertuples(name=None, index=True):
    cur.execute('''
                INSERT OR IGNORE
                INTO REVIEW VALUES(?,?,?,?,?,?,?);''', row)
    conn.commit()

## 연결 해제

In [16]:
cur.close()
conn.close()